In [1]:
from gensim.models.doc2vec import Doc2Vec

In [2]:
model = Doc2Vec.load('../models/d2v.model')

In [7]:
model.

In [12]:
import pandas as pd

articles = pd.read_csv('../src/TDS_document_topic_matrix.csv', sep='\t', index_col=0)

In [13]:
articles.article_id.head()

0    3406
1    5405
2    5957
3    6932
4    8041
Name: article_id, dtype: int64

In [19]:
model.most_similar('word'.split())

[('token', 0.4740738570690155),
 ('character', 0.4612557888031006),
 ('sentence', 0.4536151885986328),
 ('words', 0.4450315833091736),
 ('keywords', 0.40409454703330994),
 ('phrase', 0.3831368684768677),
 ('bigram', 0.37671369314193726),
 ('ingredient', 0.37507662177085876),
 ('item', 0.3664970397949219),
 ('element', 0.35849452018737793)]

In [4]:
model.docvecs.vectors

array([], shape=(0, 300), dtype=float32)

In [6]:
model.docvecs.vectors_docs.shape

(35646, 300)

In [84]:
model.total_train_time

14744.300262657915